In [5]:
# If your class is in credit_risk_models.py at project root:
from pathlib import Path
import sys
sys.path.append(str(Path.cwd().parents[1]))  # so Python can see your project files
from credit_risk_models import MertonModel

model = MertonModel(
    asset_value=25_000_000_000,   # VA
    debt_value=22_000_000_000,    # D
    risk_free_rate=0.045,         # r (4.5%)
    sigma_asset=0.35,             # σ_A (35%)
    time_to_maturity=1.0          # T (years)
)

In [6]:
import numpy as np
from math import log, sqrt

# Pull values from the model you just created
VA = model.asset_value
D = model.debt_value
r = model.risk_free_rate
sigma = model.sigma_asset
T = model.time_to_maturity

In [7]:
# d2 is Distance-to-Default
d1 = (log(VA/D) + (r + 0.5*sigma**2)*T) / (sigma*sqrt(T))
d2 = d1 - sigma*sqrt(T)

pd_1y = model.calculate_default_probability()

print(f"Distance-to-Default: {d2:.2f}σ")
print(f"PD (1y): {pd_1y:.2%}")

Distance-to-Default: 0.32σ
PD (1y): 37.49%


Executive summary

Distance-to-Default (DD): 0.32σ — the firm’s asset value sits only ~0.32 standard deviations above the default barrier at the 1-year horizon; that is a very thin cushion.

Risk-neutral PD(1y): 37.49% — default probability over one year is elevated, consistent with distressed credit. (In the Merton setup, PD ≈ Φ(−DD); Φ(−0.32) ≈ 37.5%.)

What this means economically

In the structural model, equity is a call option on assets and default occurs if assets fall below the debt due at maturity. A DD of 0.32σ implies the firm is close to the boundary: modest adverse shocks in asset value or volatility could push it into default. Compared with typical investment-grade credits (DD often > 3σ, PDs ≪ 1%), your metrics indicate high leverage and/or high asset volatility relative to the debt load and horizon.

If we illustrate with common parameters (purely for intuition): assume T=1 year, r=3%, and asset volatility σ_A=30%. Your DD=0.32 implies an “effective” log asset cushion of roughly ln(V/D) ≈ 0.11, i.e., V/D ≈ 1.12—assets only ~12% above the face of debt. Under such conditions, the hazard rate proxy is λ ≈ −ln(1−PD) ≈ 0.47, and a back-of-the-envelope “fair” one-year spread with LGD=60% is λ×LGD ≈ 28% p.a.—a distressed-level indication. (This is a risk-neutral, one-period heuristic; real-world spreads depend on liquidity, recovery, and term structure.)

Sensitivities and levers

Merton’s mechanics make the drivers explicit:

Leverage (V/D): Increasing the asset buffer raises d2 and reduces PD. Rule of thumb: a +5% rise in V lifts DD by ~ln(1.05)/(σ√T). With σ=30%, T=1, that adds ~0.16 to DD, cutting PD by ≈3.8–6 p.p. (using ∂PD/∂DD ≈ −φ(DD)).

Asset volatility (σ_A): Lower σ increases DD both by shrinking the denominator and by reducing the 0.5σ²T drag in the numerator. For the illustrative case, dropping σ from 30% to 20% (holding V/D and r constant) would raise DD from ~0.32 to ~0.61 and reduce PD from ~37% to ~27%—helpful, but still elevated.

Horizon (T): With a small DD, lengthening T often raises PD (more time for adverse diffusion) unless the expected drift in assets is sufficiently strong. Extending maturities without altering leverage/volatility doesn’t guarantee improvement in one-year PD.

Risk management implications

Capital structure: De-lever (equity raise, asset sales, junior capital, or debt buybacks) to increase V/D; avoid near-term maturities that create a “cliff” at T.

Volatility management: Hedge major risk drivers (FX, commodity, rate, concentration); prioritize stable, collateralizable cash flows; tighten working-capital variability. Lower σ_A is directly PD-reducing.

Liquidity & terming-out: Build cash buffers and refinance into longer, staggered maturities with covenants aligned to realistic volatility—this reduces barrier pressure at the horizon and execution risk.

Monitoring: Track equity-implied asset volatility and distance-to-default through time. Because PD ≈ Φ(−DD), small changes in DD around 0.3σ move PD by multiple percentage points; e.g., +0.10 in DD typically reduces PD by ~3–4 p.p.

Model considerations

Probability of default is risk-neutral. For credit risk management or rating-style metrics, consider mapping to real-world PD and extend beyond single-maturity debt (Merton treats one terminal barrier).